## **Modelos de lenguaje causales con el decodificador**


En este cuaderno, aprenderemos cómo construir y entrenar un modelo tipo GPT basado en decodificador, que es excelente para generar texto y otras tareas de procesamiento de lenguaje natural. 

### **Modelos GPT**

GPT (Generative Pretrained Transformer) es un modelo solo de decodificador porque se entrena utilizando un objetivo de modelado de lenguaje causal, donde el objetivo es predecir el siguiente token en una secuencia dados los tokens anteriores. Durante el entrenamiento, la secuencia de entrada se desplaza hacia la derecha, y el modelo aprende a generar tokens de salida de forma autoregresiva, uno a la vez. 

Este proceso permite que GPT genere texto coherente y contextualmente relevante basado en el mensaje de entrada dado. En este cuaderno aprenderemos cómo crear y entrenar un modelo tipo GPT solo de decodificador. Sin embargo, ten en cuenta que los modelos GPT reales son modelos más grandes y se entrenan con datos de entrenamiento masivos para tareas específicas de NLP.

#### **GPT vs. ChatGPT**

GPT es la familia base de modelos de lenguaje entrenados con enormes volúmenes de texto y orientados a tareas generales de NLP (generación, traducción, resumen, etc.), pero no gestionan contexto conversacional más allá del prompt inmediato.
ChatGPT, en cambio, es una versión afinada de GPT para diálogos: conserva el historial de la conversación, ajusta sus respuestas al contexto previo y genera interacciones más fluidas y coherentes en formato chatbot.

En resumen, GPT ofrece capacidades amplias de lenguaje, mientras que ChatGPT optimiza esas capacidades para mantener conversaciones continuadas y naturales.



### **Configuraciones**


#### **Instalando las librerías necesarias**


In [ ]:
#!pip install -U torchdata==0.7.1
#!pip install -Uqq portalocker>=2.0.0
#!pip install -qq torchtext==0.17.1
#!pip install -qq matplotlib
#!pip install -qq transformers

### Importando librerías requeridas


* **torchdata**: Mejora las funcionalidades de carga y preprocesamiento de datos para PyTorch, optimizando el flujo de trabajo para los modelos de aprendizaje automático.
* **portalocker**: Proporciona un mecanismo para bloquear archivos, asegurando que solo un proceso pueda acceder a un archivo a la vez, útil para gestionar recursos de archivos en aplicaciones concurrentes.
* **torchtext**: Ofrece utilidades para el procesamiento de texto y conjuntos de datos en PyTorch, simplificando la preparación de datos para tareas de procesamiento de lenguaje natural (NLP).
* **matplotlib**: Una librería de gráficos para crear visualizaciones estáticas, interactivas y animadas en Python, comúnmente utilizada para visualización de datos y creación de gráficos.

Cada una de estas librerías se utiliza para manejar diferentes aspectos de la preparación de datos, procesamiento y entrenamiento de modelos para aplicaciones de aprendizaje automático y procesamiento de lenguaje natural, mejorando el flujo de trabajo general y las capacidades del proyecto.



In [ ]:
from torchtext.datasets import multi30k, Multi30k
from torch.utils.data import DataLoader
import torch
from typing import Iterable, List
import matplotlib.pyplot as plt
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
from torchtext.vocab import Vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torchtext.datasets import IMDB,PennTreebank
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import time
from torch.optim import Adam

#Código para la supresión de warnings en el código
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

### **Pipeline de texto**

#### **Conjunto de datos**

El código carga el conjunto de datos IMDB en conjuntos de entrenamiento y validación. Luego crea un iterador para el conjunto de entrenamiento y recorre las primeras 10 muestras, imprimiendo cada una. Este proceso simula cómo uno podría iterar manualmente sobre un conjunto de datos sin usar el `DataLoader` de PyTorch para el procesamiento por lotes y la gestión de datos.

Al entrenar modelos de lenguaje, generalmente se recomienda usar texto de dominio general. Sin embargo, en este caso, estamos utilizando el conjunto de datos IMDB, que es adecuado para tareas de clasificación. No obstante, usamos IMDB debido a su tamaño reducido y compatibilidad con máquinas que tienen memoria RAM limitada. 

Para tareas de modelado de lenguaje, algunos conjuntos de datos que puedes considerar incluyen: [PennTreebank](https://pytorch.org/text/0.8.1/datasets.html#penntreebank), [WikiText-2](https://pytorch.org/text/0.8.1/datasets.html#wikitext-2), [WikiText103](https://pytorch.org/text/0.8.1/datasets.html#wikitext103)


In [ ]:
# Carga el conjunto de datos
train_iter, val_iter = IMDB()


Inicializa un iterador para el cargador de datos de entrenamiento:


In [ ]:
data_itr = iter(train_iter)  # Inicializa un iterador para el conjunto de entrenamiento

# Obtiene el tercer registro (avanzando uno por uno)
next(data_itr)  # Primer registro
next(data_itr)  # Segundo registro
next(data_itr)  # Tercer registro


Definamos el dispositivo (CPU o GPU) para el entrenamiento. 

Verificaremos si una GPU está disponible y la utilizaremos, de lo contrario usaremos la CPU.



In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

#### **Preprocesamiento de datos**

El código proporcionado se utiliza para el preprocesamiento de datos de texto, particularmente para tareas de NLP, con un enfoque en la tokenización y la construcción de vocabulario.

* **Índices y símbolos especiales**: Inicializa tokens especiales (`<unk>`, `<pad>` y una cadena vacía para EOS) con sus índices correspondientes (`0`, `1` y `2`). Estos tokens se usan para palabras desconocidas, relleno (padding) y final de oración, respectivamente.

  * `UNK_IDX`: Índice para palabras desconocidas.
  * `PAD_IDX`: Índice usado para rellenar oraciones más cortas en un lote, asegurando longitud uniforme.
  * `EOS_IDX`: Índice que representa el final de una oración (aunque no se usa explícitamente aquí, ya que el símbolo EOS se define como una cadena vacía).

* **Función `yield_tokens`**: Una función generadora que itera sobre un conjunto de datos (`data_iter`), tokeniza cada muestra de datos usando una función `tokenizer`, y produce una muestra tokenizada a la vez.

* **Construcción del vocabulario**: Construye un vocabulario a partir del conjunto de datos tokenizado. La función `build_vocab_from_iterator` procesa los tokens generados por `yield_tokens`, incluye los tokens especiales (`special_symbols`) al inicio del vocabulario y establece una frecuencia mínima (`min_freq=1`) para que los tokens sean incluidos.

* **Índice por defecto para tokens desconocidos**: Establece un índice por defecto para los tokens que no se encuentren en el vocabulario (`UNK_IDX`), asegurando que las palabras fuera de vocabulario se manejen como tokens desconocidos.

* **Función `text_to_index`**: Convierte un texto dado en una secuencia de índices basada en el vocabulario construido. Esta función es esencial para transformar texto crudo en un formato numérico que pueda ser procesado por modelos de aprendizaje automático.

* **Función `index_to_en`**: Transforma una secuencia de índices de vuelta a una cadena legible. Es útil para interpretar las salidas de los modelos y convertir predicciones numéricas nuevamente en texto.

* **Verificación de la funcionalidad**: Demuestra el uso de `index_to_en` al convertir un tensor de índices `[0,1,2]` de regreso a sus símbolos especiales correspondientes. Esto ayuda a verificar que las funciones de vocabulario y conversión de índices están funcionando como se espera.


In [ ]:
# Define los símbolos especiales y sus índices
UNK_IDX, PAD_IDX, EOS_IDX = 0, 1, 2

# Asegurarse de que los tokens estén en el orden de sus índices para insertarlos correctamente en el vocabulario
special_symbols = ['<unk>', '<pad>', '<|endoftext|>']


In [ ]:
tokenizer = get_tokenizer("basic_english")

In [ ]:
def yield_tokens(data_iter):

    for _,data_sample in data_iter:
        yield  tokenizer(data_sample)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=special_symbols, special_first=True)
vocab.set_default_index(UNK_IDX)


> **Nota: El bloque anterior debería completarse en menos de 20 segundos. Si tarda más que eso, se recomienda `reiniciar el kernel` y ejecutar las celdas posteriores a la celda que contiene los comandos `pip install` para asegurarse de que la función mencionada anteriormente opere como se espera.**


#### **Texto a índices y índice a texto**


In [ ]:
text_to_index=lambda text: [vocab(token) for token in tokenizer(text)]
index_to_en = lambda seq_en: " ".join([vocab.get_itos()[index] for index in seq_en])

In [ ]:
#Verificación
index_to_en(torch.tensor([0,1,2]))

#### **Función de colación (collate)**

En el contexto del modelo decodificador, buscamos crear una función de colación. Esta función toma un bloque de texto como entrada y produce un bloque de texto modificado como salida. La transformación del texto se logra mediante el uso de la función `get_sample(block_size, text)`. La función **get\_sample** genera una muestra aleatoria de texto (src\_sequence) y su secuencia subsiguiente (tgt\_sequence) a partir de un texto dado para el entrenamiento de modelos de lenguaje. 

Se asegura de que la muestra se ajuste al tamaño de bloque especificado y se adapta si el texto es más corto que dicho bloque, devolviendo tanto la secuencia de entrada como la de salida para el modelo.


In [ ]:
def get_sample(block_size, text):
    # Determina la longitud del texto de entrada
    sample_leg = len(text)

    # Calcula el punto de parada para seleccionar aleatoriamente una muestra
    # Esto asegura que la muestra seleccionada no exceda la longitud del texto
    random_sample_stop = sample_leg - block_size

    # Verifica si se puede tomar una muestra aleatoria (si el texto es más largo que block_size)
    if random_sample_stop >= 1:
        # Selecciona aleatoriamente un punto de inicio para la muestra
        random_start = torch.randint(low=0, high=random_sample_stop, size=(1,)).item()
        # Define el punto final de la muestra
        stop = random_start + block_size

        # Crea las secuencias de entrada y objetivo
        src_sequence = text[random_start:stop]
        tgt_sequence = text[random_start + 1:stop + 1]

    # Maneja el caso en que la longitud del texto es igual o menor al tamaño del bloque
    elif random_sample_stop <= 0:
        # Comienza desde el inicio y usar todo el texto disponible
        random_start = 0
        stop = sample_leg
        src_sequence = text[random_start:stop]
        tgt_sequence = text[random_start + 1:stop]
        # Añade una cadena vacía para mantener la alineación de secuencias
        tgt_sequence.append('<|endoftext|>')

    return src_sequence, tgt_sequence


Probemos primero `get_sample(block_size, text)` y obtengamos un lote de textos:


In [ ]:
BATCH_SIZE=1

batch_of_tokens=[]

for i in range(BATCH_SIZE):
  _,text =next(iter(train_iter))
  batch_of_tokens.append(tokenizer(text))

Obtenemos la primera muestra de texto:


In [ ]:
text=batch_of_tokens[0][0:100]
text[0:100]
batch_of_tokens

Para probar la función `get_sample` con un tamaño de bloque de 100, donde la salida incluye tanto la secuencia fuente como la secuencia objetivo, siendo la secuencia objetivo la secuencia fuente desplazada en un carácter, podemos usar el siguiente código como ejemplo:



In [ ]:
block_size=10
src_sequences, tgt_sequence=get_sample( block_size, text)

Verificamos si la secuencia está desplazada.



In [ ]:
print("src: ",src_sequences)
print("tgt: ",tgt_sequence)

El siguiente código crea lotes de secuencias fuente (`src_batch`) y objetivo (`tgt_batch`) a partir de un conjunto de datos para entrenar modelos de NLP. 

Recorre el conjunto de datos para extraer muestras de texto, genera las secuencias fuente y objetivo correspondientes usando la función `get_sample`, las convierte en índices del vocabulario y luego en tensores de PyTorch. 

Cada iteración agrega estas secuencias a sus respectivas listas de lotes y muestra sus detalles, incluyendo el texto, los índices y las dimensiones de los tensores, para dos muestras por lote.


In [ ]:
# Inicializa listas vacías para almacenar las secuencias fuente y objetivo
src_batch, tgt_batch = [], []

# Define el tamaño del lote
BATCH_SIZE = 2

# Bucle para crear lotes de secuencias fuente y objetivo
for i in range(BATCH_SIZE):
    # Obtiene el siguiente dato del iterador de entrenamiento
    _, text = next(iter(train_iter))

    # Genera las secuencias fuente y objetivo usando la función get_sample
    src_sequence_text, tgt_sequence_text = get_sample(block_size, tokenizer(text))

    # Convierte las secuencias fuente y objetivo a índices del vocabulario tokenizado
    src_sequence_indices = vocab(src_sequence_text)
    tgt_sequence_indices = vocab(tgt_sequence_text)

    # Convierte las secuencias a tensores de PyTorch con tipo de dato int64
    src_sequence = torch.tensor(src_sequence_indices, dtype=torch.int64)
    tgt_sequence = torch.tensor(tgt_sequence_indices, dtype=torch.int64)

    # Agrega las secuencias fuente y objetivo a sus respectivos lotes
    src_batch.append(src_sequence)
    tgt_batch.append(tgt_sequence)

    # Imprime la salida para cada muestra (en este caso, cada segunda muestra)
    print(f"Muestra {i}:")
    print("Secuencia fuente (texto):", src_sequence_text)
    print("Secuencia fuente (índices):", src_sequence_indices)
    print("Secuencia fuente (forma):", src_sequence.shape)
    print("Secuencia objetivo (texto):", tgt_sequence_text)
    print("Secuencia objetivo (índices):", tgt_sequence_indices)
    print("Secuencia objetivo (forma):", tgt_sequence.shape)


La función `collate_batch` prepara lotes de secuencias fuente y objetivo para el entrenamiento procesando cada muestra de texto en un lote dado. Genera las secuencias fuente y objetivo utilizando la función `get_sample` con un tamaño de bloque especificado, convierte estas secuencias en índices usando un vocabulario y las transforma en tensores de PyTorch. 

Luego, las secuencias se rellenan (padding) para asegurar una longitud uniforme en todo el lote. Finalmente, devuelve los lotes fuente y objetivo ya rellenados, listos para el entrenamiento en el dispositivo especificado (`DEVICE`).


In [ ]:
BLOCK_SIZE=30
def collate_batch(batch):
    src_batch, tgt_batch = [], []
    for _,_textt in batch:
      src_sequence,tgt_sequence=get_sample(BLOCK_SIZE,tokenizer(_textt))
      src_sequence=vocab(src_sequence)
      tgt_sequence=vocab(tgt_sequence)
      src_sequence= torch.tensor(src_sequence, dtype=torch.int64)
      tgt_sequence = torch.tensor(tgt_sequence, dtype=torch.int64)
      src_batch.append(src_sequence)
      tgt_batch.append(tgt_sequence)


    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=False)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=False)

    return src_batch.to(DEVICE), tgt_batch.to(DEVICE)

El código configura cargadores de datos para los conjuntos de entrenamiento, validación y prueba utilizando la clase `DataLoader`, donde cada conjunto utiliza una función personalizada `collate_batch` para el procesamiento por lotes. Los cargadores de datos manejan lotes de tamaño 1 por simplicidad y mezclan aleatoriamente los datos para un acceso aleatorio. Después de inicializar el cargador de datos de entrenamiento, se obtiene el primer lote de secuencias fuente (`src`) y objetivo (`tgt`). 

Luego, itera sobre cada token en la secuencia fuente, los convierte nuevamente a texto usando la función `index_to_en` y muestra las oraciones resultantes, demostrando cómo acceder y visualizar los datos preprocesados que están listos para el entrenamiento del modelo.


In [ ]:
BATCH_SIZE=1
dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
val_dataloader= DataLoader(val_iter , batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

#### **Iterando a través de muestra de datos**
El código proporcionado itera a través de lotes de pares fuente-objetivo desde un cargador de datos. Demuestra cómo acceder y mostrar algunas muestras del conjunto de datos:

* Se inicializa un iterador sobre el cargador de datos llamado `dataset`.
* Un bucle se ejecuta durante 10 iteraciones para obtener y mostrar los primeros 10 pares fuente-objetivo. Para cada par:

  * `src` y `trt` (abreviatura de target) contienen el lote de secuencias fuente y objetivo respectivamente.
  * La función `index_to_en` se utiliza para convertir estas secuencias de índices numéricos a texto legible.
  * Se imprime el número de `sample` y los textos fuente y objetivo correspondientes.

Después de imprimir las primeras 10 muestras, el código continúa iterando a través del conjunto de datos:

* Se imprime la forma (shape) de los tensores objetivo y fuente del siguiente lote, lo que proporciona información sobre el número de tokens y el tamaño del lote.
* Nuevamente se utiliza la función `index_to_en` para convertir la primera secuencia del lote de índices a texto tanto para la fuente como para el objetivo.
* Solo se imprime el primer par de los lotes restantes y luego el bucle se rompe.

Este proceso es útil para verificar que el cargador de datos esté funcionando correctamente y que las secuencias se estén transformando adecuadamente.


In [ ]:
dataset=iter(dataloader)
for sample in range(10):
  src,trt=next(dataset)
  print("Muestra",sample)
  print("Fuente:",index_to_en(src))
  print("\n")
  print("Objetivo:",index_to_en(trt))
  print("\n")

In [ ]:
for  src,trt in dataset:
    print(trt.shape)
    print(src.shape)
    print(index_to_en(src[0,:]))
    print(index_to_en(trt[0,:]))
    break

Asegúrate de que la secuencia fuente y la secuencia objetivo estén desplazadas.


In [ ]:
print("Fuente:",index_to_en(src))
print("Objetivo:",index_to_en(trt))

Ahora que hemos cubierto la preparación de los datos, pasemos a comprender los componentes clave del modelo Transformer.



#### **Enmascaramiento**

En los transformers, el enmascaramiento es crucial para asegurar que ciertas posiciones no sean atendidas. La función `generate_square_subsequent_mask` produce una matriz triangular superior, lo que garantiza que durante la decodificación, un token no pueda atender a tokens futuros del objetivo.

In [ ]:
def generate_square_subsequent_mask(sz,device=DEVICE):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

La función `create_mask`, por otro lado, genera máscaras para la secuencia fuente, basándose en la secuencia fuente proporcionada.


In [ ]:
def create_mask(src,device=DEVICE):
    src_seq_len = src.shape[0]
    src_mask = generate_square_subsequent_mask(src_seq_len)
    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    return src_mask,src_padding_mask

Veamos un ejemplo de un tensor fuente y sus máscaras asociadas:


In [ ]:
# Reemplaza los primeros cuatro tokens con el token PAD para verificar cómo los tokens de relleno (padding) son enmascarados usando padding_mask
src[0:4] = PAD_IDX

In [ ]:
mask,padding_mask = create_mask(src)
src

In [ ]:
mask

In [ ]:
padding_mask

#### **Codificación posicional**

El modelo Transformer no tiene conocimiento incorporado del orden de los tokens en la secuencia. Para proporcionarle esta información al modelo, se agregan codificaciones posicionales a los embeddings de los tokens. Estas codificaciones siguen un patrón fijo basado en su posición dentro de la secuencia.

GPT utiliza codificaciones posicionales entrenables. A diferencia de las codificaciones posicionales fijas (como las codificaciones sinusoidales utilizadas en el artículo original de Transformer), las codificaciones posicionales entrenables se aprenden durante el proceso de entrenamiento del modelo.

Las codificaciones posicionales entrenables se implementan como un conjunto de parámetros aprendibles, uno para cada posición en la secuencia de entrada. Estos parámetros tienen la misma dimensionalidad que los embeddings de los tokens. Durante el entrenamiento, el modelo actualiza los parámetros de codificación posicional junto con los demás parámetros del modelo para capturar la información posicional de manera más efectiva.

El uso de codificaciones posicionales entrenables en GPT permite al modelo aprender representaciones posicionales más flexibles y específicas para la tarea, lo que potencialmente mejora su rendimiento en diversas tareas de procesamiento de lenguaje natural.

En el contexto de este cuadernos, nos mantenemos con la codificación posicional fija por simplicidad.


In [ ]:
# agrega información posicional a los tokens de entrada
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

#### **Embeddings de tokens**

Los embeddings de tokens, o embeddings de palabras o representación de palabras, es una forma de convertir palabras o tokens de un corpus de texto en vectores numéricos dentro de un espacio vectorial continuo. A cada palabra o token único en el corpus se le asigna un vector de longitud fija, donde los valores numéricos representan diversas propiedades lingüísticas de la palabra, como su significado, contexto o relaciones con otras palabras.

La clase `TokenEmbedding` que se muestra a continuación convierte tokens numéricos en embeddings:


In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

#### **Arquitectura personalizada GPT**

La clase `CustomGPTModel` define una arquitectura de modelo basada en transformer para modelos generativos preentrenados. Este modelo tiene como objetivo generar texto y realizar diversas tareas de NLP. A continuación se presenta una explicación de los principales componentes de la clase:

* **Initialización (`__init__`)**: El constructor toma varios parámetros incluyendo `embed_size`, `vocab_size`, `num_heads`, `num_layers`, `max_seq_len` y `dropout`. Inicializa la capa embedding, la codificación posicional, las capas codificadoras del transformer y una capa lineal (`lm_head`) para generar los logits sobre el vocabulario.

* **Inicialización de pesos (`init_weights`)**: Este método inicializa los pesos del modelo para una mejor convergencia durante el entrenamiento. Se utiliza la inicialización uniforme de Xavier, que es una práctica común para inicializar pesos en aprendizaje profundo.

* **Decoder (`decoder`)**: Aunque se llama `decoder`, este método actualmente funciona como el paso hacia adelante (forward pass) a través de las capas codificadoras del transformer, seguido por la generación de logits para la tarea de modelado del lenguaje. Se encarga de añadir las codificaciones posicionales a los embeddings y aplica una máscara si es necesario.

* **Forward pass (`forward`)**: Este método es similar al método `decoder` y define el cálculo hacia adelante del modelo. Procesa la entrada a través de las capas de embedding, codificación posicional, capas codificadoras del transformer, y produce la salida final usando `lm_head`.

* **Generación de máscaras**: Tanto los métodos `decoder` como `forward` contienen lógica para generar una máscara causal cuadrada si no se proporciona una máscara fuente. Esta máscara asegura que la predicción para una posición no dependa de los tokens futuros en la secuencia, lo cual es importante para la naturaleza autoregresiva de los modelos GPT.

Una sección del código está comentada, lo que sugiere un diseño inicial en el que se consideró una capa decodificadora de transformer. Sin embargo, la implementación final utiliza solo capas codificadoras, lo cual es una simplificación común para modelos enfocados en modelado y generación de lenguaje.

Esta clase encapsula eficazmente los componentes necesarios para crear un modelo tipo GPT, permitiendo su entrenamiento en tareas de modelado de lenguaje y aplicaciones de generación de texto.


In [ ]:
class CustomGPTModel(nn.Module):
    def __init__(self, embed_size, vocab_size, num_heads, num_layers, max_seq_len=500, dropout=0.1):

        super().__init__()

        # Inicializa los pesos del modelo
        self.init_weights()

        # Capa de embeddings de tokens
        self.embed = nn.Embedding(vocab_size, embed_size)

        # Codificación posicional para proporcionar información de orden en la secuencia
        self.positional_encoding = PositionalEncoding(embed_size, dropout=dropout)

        print(embed_size)

        # Las capas restantes forman parte del codificador Transformer
        encoder_layers = nn.TransformerEncoderLayer(d_model=embed_size, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)
        self.embed_size = embed_size

        # Capa lineal final para proyectar a logits sobre el vocabulario
        self.lm_head = nn.Linear(embed_size, vocab_size)

    def init_weights(self):
        # Inicialización de pesos con Xavier para mejor convergencia
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def create_mask(src, device=DEVICE):
        # Crea una máscara causal y una máscara de padding para la entrada
        src_seq_len = src.shape[0]
        src_mask = nn.Transformer.generate_square_subsequent_mask(src_seq_len)
        src_padding_mask = (src == PAD_IDX).transpose(0, 1)
        return src_mask, src_padding_mask

    def decoder(self, x, src_mask):
        seq_length = x.size(0)

        # Agrega codificaciones posicionales a los embeddings de entrada
        x = self.embed(x) * math.sqrt(self.embed_size)
        x = self.positional_encoding(x)

        # Si no se proporciona una máscara fuente, generar una máscara causal
        if src_mask is None:
            """Genera una máscara causal cuadrada para la secuencia. Las posiciones enmascaradas se llenan con float('-inf').
            Las posiciones no enmascaradas se llenan con float(0.0).
            """
            src_mask, src_padding_mask = create_mask(x)

        # Pasa por las capas del codificador Transformer
        output = self.transformer_encoder(x, src_mask)

        # Proyecta las salidas a logits del vocabulario
        logits = self.lm_head(x)
        return logits

    def forward(self, x, src_mask=None, key_padding_mask=None):
        seq_length = x.size(0)

        # Agrega codificaciones posicionales a las embedding de entrada
        x = self.embed(x) * math.sqrt(self.embed_size)  # src = self.embedding(src) * math.sqrt(self.d_model)
        x = self.positional_encoding(x)

        # Si no se proporciona una máscara fuente, generar una máscara causal
        if src_mask is None:
            """Genera una máscara causal cuadrada para la secuencia. Las posiciones enmascaradas se llenan con float('-inf').
            Las posiciones no enmascaradas se llenan con float(0.0).
            """
            src_mask, src_padding_mask = create_mask(x)

        # Pasa por el codificador Transformer
        output = self.transformer_encoder(x, src_mask, key_padding_mask)

        # Proyecta las salidas a logits del vocabulario
        x = self.lm_head(x)

        return x


#### **Configuración e inicialización del modelo**

Aquí configuramos e instanciamos un modelo GPT personalizado con las siguientes especificaciones:

* `ntokens`: El número total de tokens únicos en el vocabulario, que el modelo utilizará para representar palabras.
* `emsize`: El tamaño de cada vector de embeddings. En este modelo, cada palabra será representada por un vector de 200 dimensiones.
* `nlayers`: El número de capas codificadoras del transformer en el modelo. Estamos utilizando dos capas en esta configuración.
* `nhead`: El número de cabeceras de atención en el mecanismo de atención multi-cabecera. El modelo usará dos cabeceras de atención.
* `dropout`: Una técnica de regularización en la que se ignoran aleatoriamente algunas neuronas durante el entrenamiento para prevenir el sobreajuste. Aquí, establecemos la probabilidad de dropout en 0.2.

Después de definir estos hiperparámetros, creamos una instancia de `CustomGPTModel` pasando el tamaño de los embeddings, el número de cabeceras de atención, el número de capas, el tamaño del vocabulario y la probabilidad de dropout. Luego, el modelo se mueve al `DEVICE` especificado, que puede ser una CPU o una GPU, para su entrenamiento o inferencia.


In [ ]:
ntokens = len(vocab)  # tamaño del vocabulario
emsize = 200  # dimensión de los embeddings
nlayers = 2  # número de capas ``nn.TransformerEncoderLayer`` en ``nn.TransformerEncoder``
nhead = 2  # número de cabeceras en ``nn.MultiheadAttention``
dropout = 0.2  # probabilidad de dropout

# Crea el modelo personalizado GPT y moverlo al dispositivo (CPU o GPU)
modelo = CustomGPTModel(embed_size=emsize, num_heads=nhead, num_layers=nlayers, vocab_size=ntokens, dropout=dropout).to(DEVICE)


### **Generación con prompt (prompting)**
Para que el modelo genere texto (el siguiente token), necesitas crear un punto de inicio, al que llamamos *prompt*, para que el modelo agregue tokens y genere texto a partir de él. 

Debes verificar que el *prompt* no sea `None` ni demasiado largo, luego procede a tokenizarlo, convertirlo en índices y reestructurarlo según sea necesario.



In [ ]:
def encode_prompt(prompt, block_size=BLOCK_SIZE):
    # Maneja el caso en que el prompt sea None
    while prompt is None:
        prompt = input("Lo siento, el prompt no puede estar vacío. Por favor, ingresa un prompt válido: ")

    # Tokeniza el prompt
    tokens = tokenizer(prompt)
    number_of_tokens = len(tokens)

    # Maneja el caso de prompts muy largos
    if number_of_tokens > block_size:
        tokens = tokens[-block_size:]  # Conserva solo los últimos block_size tokens

    # Convierte los tokens a índices del vocabulario
    prompt_indices = vocab(tokens)

    # Convierte los índices a un tensor de PyTorch y reestructurarlo
    prompt_encoded = torch.tensor(prompt_indices, dtype=torch.int64).reshape(-1, 1)

    return prompt_encoded


Veamos algunos ejemplos diferentes donde la entrada es `None` o más larga que el tamaño del bloque (`block size`):



In [ ]:
print(index_to_en(encode_prompt(None)))

In [ ]:
print(index_to_en(encode_prompt("Este es un prompt para que el modelo genere las siguientes palabras." ) ))

Ahora, codifiquemos un *prompt* de texto y ejecutémoslo a través de la parte decodificadora del modelo:

* Se llama al método `decoder` de la instancia `modelo` de `CustomGPTModel` con el *prompt* codificado y sin una máscara de entrada (`src_mask=None`), lo que indica que no se enmascarará ninguna parte de la secuencia durante el procesamiento. El decodificador se encargará de crear una máscara causal internamente si es necesario.
* La salida `logits` representa las predicciones en bruto del modelo para cada posición del token, las cuales pueden procesarse posteriormente (por ejemplo, aplicando una función *softmax*) para obtener las probabilidades del siguiente token en la secuencia.


In [ ]:
prompt_encoded=encode_prompt("Este es un prompt para que el modelo genere las siguientes palabras.").to(DEVICE)
prompt_encoded

In [ ]:
logits = modelo.decoder(prompt_encoded,src_mask=None).to(DEVICE)

Tenemos 11 tokens por salida, una dimensión adicional de lote (*batch*), junto con los valores de *logits* correspondientes para cada palabra en el vocabulario.



In [ ]:
logits.shape

Reestructuramos de manera que la dimensión del lote (*batch*) sea cinco.



In [ ]:
logits = logits.transpose(0, 1)
logits.shape

Logits contiene los logits para cada token en la secuencia generada por el decodificador, solo necesitamos el último para la siguiente palabra.



In [ ]:
logit_preiction =logits[:,-1]
logit_preiction.shape

Obtiene el índice de la siguiente palabra.



In [ ]:
 _, next_word_index = torch.max(logit_preiction, dim=1)
 next_word_index

Próxima palabra


In [ ]:
index_to_en(next_word_index)

### **Generación autoregresiva de texto**

En los modelos de decodificador, simplemente se añade la salida a la entrada para generar la siguiente respuesta. Este proceso se detiene cuando se encuentra la etiqueta de fin de secuencia `<|endoftext|>` o si la entrada se vuelve demasiado grande. Más adelante en este notebook, implementaremos esto como una función.


In [ ]:
prompt="este es el inicio de"

Asegurate de que el *prompt* tenga el tamaño máximo de entrada y realizar una predicción.



In [ ]:
prompt_encoded = encode_prompt(prompt).to(DEVICE)
print("Dispositivo para prompt_encoded:", prompt_encoded.shape)

In [ ]:
max_new_tokens=10

In [ ]:
for i in range(max_new_tokens):
    # Obtiene los logits del modelo a partir del prompt codificado
    logits = modelo.decoder(prompt_encoded, src_mask=None)
    logits = logits.transpose(0, 1)

    print(" ")
    print(f"Forma (shape) de los logits en el paso {i}: {logits.shape}")

    # Obtiene los logits de la última posición (último token)
    logit_preiction = logits[:, -1]
    print(f"Forma de logit_prediction en el paso {i}: {logit_preiction.shape}")

    # Obtiene el índice del token con mayor probabilidad (token siguiente)
    next_token_encoded = torch.argmax(logit_preiction, dim=-1).reshape(-1, 1)
    print(f"Forma de next_token_encoded en el paso {i}: {next_token_encoded.shape}")

    # Añade el nuevo token predicho al final de la secuencia
    prompt_encoded = torch.cat((prompt_encoded, next_token_encoded), dim=0).to(DEVICE)
    print(f"Secuencia en el paso {i}: {[index_to_en(j) for j in prompt_encoded]}")
    print(f"Forma de prompt_encoded después de la concatenación en el paso {i}: {prompt_encoded.shape}")


Ahora vamos a implementarlo como una función.



In [ ]:
# Define los símbolos especiales y sus índices
UNK_IDX, PAD_IDX, EOS_IDX = 0, 1, 2

# Asegurate de que los tokens estén en el orden de sus índices para insertarlos correctamente en el vocabulario
special_symbols = ['<unk>', '<pad>', '<|endoftext|>']

BLOCK_SIZE


In [ ]:
# Generación de texto autoregresiva con un modelo de lenguaje
def generate(modelo, prompt=None, max_new_tokens=500, block_size=BLOCK_SIZE, vocab=vocab, tokenizer=tokenizer):
    # Mueve el modelo al dispositivo especificado (por ejemplo, GPU o CPU)
    modelo.to(DEVICE)

    # Codifica el prompt de entrada usando la función encode_prompt
    prompt_encoded = encode_prompt(prompt).to(DEVICE)
    tokens = []

    # Genera nuevos tokens hasta alcanzar el máximo especificado
    for _ in range(max_new_tokens):
        # Decodifica el prompt codificado utilizando el decodificador del modelo
        logits = modelo(prompt_encoded, src_mask=None, key_padding_mask=None)

        # Transpone los logits para poner la longitud de la secuencia como la primera dimensión
        logits = logits.transpose(0, 1)

        # Selecciona los logits del último token de la secuencia
        logit_prediction = logits[:, -1]

        # Escoge el token más probable a partir de los logits (decodificación codiciosa)
        next_token_encoded = torch.argmax(logit_prediction, dim=-1).reshape(-1, 1)

        # Si el siguiente token es el token de fin de secuencia (EOS), detener la generación
        if next_token_encoded.item() == EOS_IDX:
            break

        # Añade el siguiente token al prompt codificado y conservar solo los últimos 'block_size' tokens
        prompt_encoded = torch.cat((prompt_encoded, next_token_encoded), dim=0)[-block_size:]

        # Convierte el índice del siguiente token en una cadena usando el vocabulario
        # Mueve el tensor de nuevo a la CPU para la búsqueda en el vocabulario si es necesario
        token_id = next_token_encoded.to('cpu').item()
        tokens.append(vocab.get_itos()[token_id])

    # Une los tokens generados en una sola cadena y devolverla
    return ' '.join(tokens)


In [ ]:
generate(modelo,prompt="este es el inicio de ",max_new_tokens=30,vocab=vocab,tokenizer=tokenizer)

### **Decodificando las diferencias: Entrenamiento vs. inferencia**

La diferencia clave entre las etapas de entrenamiento e inferencia radica en las entradas al decodificador. Durante el entrenamiento, el decodificador se beneficia de tener acceso a los *ground truth*, recibiendo los tokens exactos de la secuencia objetivo de forma incremental a través de una técnica conocida como **"teacher forcing"**. Este enfoque contrasta notablemente con otras arquitecturas de redes neuronales que dependen de las predicciones previas de la red como entradas durante el entrenamiento. Una vez concluido el entrenamiento, los conjuntos de datos utilizados se asemejan a los empleados en modelos de redes neuronales más convencionales, proporcionando una base familiar para la comparación y evaluación.

Para iniciar el entrenamiento, primero se debe crear un objeto de pérdida de entropía cruzada (*Cross Entropy Loss*). La función de pérdida no tomará en cuenta los tokens de relleno (*PAD*).


In [ ]:
from torch.nn import CrossEntropyLoss
loss_fn = CrossEntropyLoss(ignore_index=PAD_IDX)

Creamos las máscaras requeridas


In [ ]:
src,tgt=next(iter(dataloader))

mask,padding_mask = create_mask(src)

Cuando llamas a `modelo(src, src_mask, key_padding_mask)`, el método `forward` de la clase `CustomGPTModel` genera los *logits* para la secuencia objetivo, los cuales luego pueden traducirse en tokens reales tomando la predicción con la mayor probabilidad en cada paso de la secuencia.


In [ ]:
logits = modelo(src,src_mask=mask,key_padding_mask=padding_mask)
print(logits.shape)

In [ ]:
print("forma de salida (output shape)", logits.shape)
print("forma de la secuencia fuente (source shape)", src)


Durante el entrenamiento, al decodificador del transformer se le proporciona toda la secuencia objetivo de una sola vez. Esto permite el procesamiento paralelo de la secuencia, a diferencia de la generación de un token a la vez. En consecuencia, la secuencia de salida se produce en su totalidad, coincidiendo con la forma (shape) de la secuencia objetivo de entrada. Esta generación paralela es eficiente y aprovecha la capacidad del modelo para manejar secuencias de manera integral. Al examinar las dimensiones de la salida, podemos confirmar que se alinean con la secuencia objetivo de entrada, lo que indica que toda la secuencia ha sido procesada simultáneamente.



Eliminamos la primera muestra de la secuencia objetivo.



In [ ]:
tgt
print(tgt.shape)

In [ ]:
print(logits.reshape(-1, logits.shape[-1]).shape)
print(tgt.reshape(-1).shape)

Ahora calculamos la pérdida, ya que la salida del decodificador del transformer se proporciona como entrada a la función de pérdida de entropía cruzada junto con los valores de la secuencia objetivo. Dado que la salida del transformer tiene las dimensiones de longitud de secuencia, tamaño del lote (*batch size*) y características (*features*), es necesario reorganizar (*reshape*) esta salida para que coincida con el formato estándar requerido por la función de pérdida de entropía cruzada. Este paso asegura que la pérdida se calcule correctamente, comparando la secuencia predicha con las *ground truth* en cada paso temporal a lo largo del lote, utilizando el método `reshape`.


In [ ]:
loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
print(loss.item())

Siguiendo los procedimientos mencionados anteriormente, podemos desarrollar una función capaz de realizar predicciones y, posteriormente, calcular la pérdida correspondiente sobre los datos de validación. Utilizaremos esta función más adelante.


In [ ]:
def evaluate(modelo: nn.Module, eval_data) -> float:
    modelo.eval()  
    total_loss = 0.
    with torch.no_grad():
        for src,tgt in eval_data:
            tgt = tgt.to(DEVICE)
            #seq_len = src.size(0)
            logits = modelo(src,src_mask=None,key_padding_mask=None)
            total_loss +=  loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1)).item()
    return total_loss / (len(list(eval_data)) - 1)

In [ ]:
evaluate(modelo,val_dataloader)

#### Entrenamiento del modelo

Incorporando los pasos descritos anteriormente, procedemos a entrenar el modelo. Aparte de estos procedimientos específicos, el proceso general de entrenamiento sigue los métodos convencionales empleados en el entrenamiento de redes neuronales.

**Ten en cuenta que entrenar el modelo utilizando CPUs puede ser un proceso que consume mucho tiempo. Si no tienes acceso a GPUs, puedes saltar y continuar cargando el modelo preentrenado usando el código proporcionado. Hemos entrenado el modelo durante 30 épocas y lo hemos guardado para tu conveniencia.**

La función `train` está definida para ajustar finamente el `CustomGPTModel` sobre un conjunto de datos de entrenamiento dado. Está estructurada de la siguiente manera:

* **Optimizador**: Se inicializa un optimizador ADAM.

Dentro de la función `train`:

* El modelo se establece en modo entrenamiento, lo que habilita las capas de *dropout* y de *batch normalization*.
* Un bucle itera sobre los datos de entrenamiento, que se cargan en lotes. Para cada lote:

  * Se extraen las secuencias fuente (`src`) y objetivo (`tgt`).
  * El modelo realiza una pasada hacia adelante (*forward pass*) para obtener los *logits*.
  * Los *logits* se reestructuran para el cálculo de la pérdida.
  * La pérdida se calcula utilizando `loss_fn`, que probablemente se refiere a una función de pérdida como la entropía cruzada, que mide la diferencia entre los *logits* predichos y las secuencias objetivo.
* Se aplica recorte de gradientes (*gradient clipping*) para prevenir gradientes explosivos, lo cual es común en el entrenamiento de redes neuronales profundas.
* El optimizador actualiza los parámetros del modelo basándose en los gradientes calculados.

El registro (*logging*) ocurre cada `10000` pasos, o al alcanzar un lote específico (el lote `42060` está codificado como ejemplo). Durante el registro:

* Se calculan e imprimen la pérdida promedio y la *perplejidad* (una medida de qué tan bien el modelo de probabilidad predice una muestra), lo que proporciona información sobre el rendimiento del modelo.
* Se mide e informa el tiempo transcurrido por lote desde el último intervalo de registro, lo que da una indicación de la eficiencia del entrenamiento.


In [ ]:
optimizer = Adam(modelo.parameters(), lr=1e-2, weight_decay=0.01, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10000, gamma=0.9)

def train(modelo: nn.Module,train_data) -> None:
    modelo.train()  # modo de entrenamiento
    total_loss = 0.
    log_interval = 10000
    start_time = time.time()

    num_batches = len(list(train_data)) // block_size
    for batch,srctgt in enumerate(train_data):
        src= srctgt[0]
        tgt= srctgt[1]
        logits = modelo(src,src_mask=None)
        logits_flat = logits.reshape(-1, logits.shape[-1])
        loss = loss_fn(logits_flat, tgt.reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(modelo.parameters(), 0.5)
        optimizer.step()
        total_loss += loss.item()

        if (batch % log_interval == 0 and batch > 0) or batch==42060:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            #cur_loss = total_loss / log_interval
            cur_loss = total_loss / batch
            ppl = math.exp(cur_loss)
            print(f'| epoca {epoch:3d} | {batch//block_size:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.4f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            start_time = time.time()

    return total_loss

Usamos listas de pérdidas para hacer un seguimiento de la pérdida de entrenamiento y validación.

El modelo recorrerá los datos de entrenamiento 30 veces (épocas). Este paso de entrenamiento utiliza funciones que hemos definido anteriormente.

In [ ]:
best_val_loss = float('inf')
epochs = 30
Train_losses= []
Val_losses = []
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train_loss = train(modelo,dataloader)
    val_loss = evaluate(modelo, val_dataloader)
    val_ppl = math.exp(val_loss)
    Train_losses.append(train_loss)
    Val_losses.append(val_loss)

    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| Fin de epoca {epoch:3d} | tiempo: {elapsed:5.2f}s | '
        f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(modelo.state_dict(), 'model_best_val_loss.pt')

Vamos a graficar las pérdidas de entrenamiento y validación:


In [ ]:
# Calcula el número de épocas (suponiendo que las longitudes de Train_losses y Val_losses sean iguales)
num_epochs = len(Train_losses)

# Crea una figura y un conjunto de subgráficas
fig, ax = plt.subplots()

# Grafica las pérdidas de entrenamiento
ax.plot(range(num_epochs), Train_losses, label='Pérdida de entrenamiento', color='blue')

# Grafica las pérdidas de validación
ax.plot(range(num_epochs), Val_losses, label='Pérdida de validación', color='orange')

# Establece la etiqueta del eje x
ax.set_xlabel('Época')

# Establece la etiqueta del eje y
ax.set_ylabel('Pérdida')

# Establece el título del gráfico
ax.set_title('Pérdidas de entrenamiento y validación')

# Añade una leyenda al gráfico
ax.legend()

# Muestra el gráfico
plt.show()

#### Cargando el modelo guardado

Si deseas omitir el entrenamiento y cargar un modelo entrenado que hemos proporcionado, adelante, descomenta la siguiente celda:


In [ ]:
#!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/kyn1_OsXrzjef0xihlsXmg.pt'
#modelo.load_state_dict(torch.load('kyn1_OsXrzjef0xihlsXmg.pt',map_location=torch.device('cpu')))

In [ ]:
print(generate(modelo,prompt="the movie was",max_new_tokens=10,vocab=vocab,tokenizer=tokenizer))

Puedes ver que el resultado no es satisfactorio, lo cual se debe al hecho de que los LLMs necesitan ser entrenados con grandes volúmenes de datos durante varias épocas para ser precisos.


### **Cargando el modelo GPT2 desde HuggingFace**

Ahora carguemos el modelo GPT2 desde HuggingFace para comprobar cómo se desempeña en la generación de texto:


In [ ]:
# Carga el tokenizador y el modelo
tokenizer1 = GPT2Tokenizer.from_pretrained("gpt2")
modelo = GPT2LMHeadModel.from_pretrained("gpt2")

# Define el texto de entrada (prompt)
#input_text = "Once upon a time in a faraway land,"
input_text = "the movie was"

# Tokeniza el texto de entrada y prepararlo para el modelo
input_ids = tokenizer1.encode(input_text, return_tensors="pt")

# Genera texto usando el modelo
# Establece la longitud deseada del texto generado (max_length),
# y otros parámetros de generación como temperature, top_k y top_p
max_length = 15
temperature = 0.7
top_k = 50
top_p = 0.95

generated_ids = modelo.generate(
    input_ids,
    max_length=max_length,
    temperature=temperature,
    top_k=top_k,
    top_p=top_p,
    pad_token_id=tokenizer1.eos_token_id,
)

# Decodifica el texto generado
generated_text = tokenizer1.decode(generated_ids[0], skip_special_tokens=True)

# Imprimir el texto de entrada y el texto generado
print(f"Entrada: {input_text}")
print(f"Texto generado: {generated_text}")


#### **Preguntas**


1. ¿Qué papel juegan `UNK_IDX`, `PAD_IDX` y `EOS_IDX` en el vocabulario y por qué es importante definirlos antes de construirlo?
2. ¿Cómo funciona la función `yield_tokens` y por qué usamos un generador para construir el vocabulario?
3. ¿Qué efecto tiene `special_first=True` al llamar a `build_vocab_from_iterator`?
4. Explica paso a paso qué hace `text_to_index` y cómo se relaciona con `index_to_en`.
5. En `get_sample`, ¿cómo se determina `random_sample_stop` y qué casos cubre cada rama del `if`?
6. ¿Por qué en el caso de `random_sample_stop <= 0` se añade explícitamente `'<|endoftext|>'` al final de `tgt_sequence`?
7. ¿Qué resultado obtenemos al concatenar varias cadenas de tokens con `pad_sequence` en el `collate_batch`?
8. Explica la diferencia entre usar `batch_first=False` y `batch_first=True` en `pad_sequence`.
9. ¿Cómo genera `generate_square_subsequent_mask` una máscara causal y por qué la transposición de índices es necesaria?
10. ¿Qué información codifica `padding_mask` y cómo mejora la atención en el Transformer?
11. En la clase `PositionalEncoding`, ¿por qué se usan funciones seno y coseno con frecuencias crecientes?
12. ¿Qué ventaja aporta multiplicar el embedding por `sqrt(self.emb_size)` en `TokenEmbedding`?
13. ¿Cuál es la función de `init_weights` y por qué se prefiere la inicialización de Xavier para pesos de más de una dimensión?
14. Analiza el método `decoder` de `CustomGPTModel`: ¿qué pasos realiza desde la entrada hasta la proyección final en `lm_head`?
15. ¿Por qué el método `forward` de `CustomGPTModel` requiere tanto `src_mask` como `key_padding_mask`?
16. En la función `encode_prompt`, ¿cómo se manejan los prompts más largos que `block_size` y por qué?
17. Describe cómo funciona la generación autoregresiva en el bucle de `generate`, especialmente la lógica de truncar a los últimos `block_size` tokens.
18. ¿Cómo se computa la pérdida con `CrossEntropyLoss(ignore_index=PAD_IDX)` y qué papel juega el parámetro `ignore_index`?
19. En la rutina de entrenamiento (`train`), ¿por qué se llama a `clip_grad_norm_` y qué problema evita?
20. ¿Qué métrica de evaluación representa `ppl` (perplejidad) y cómo se calcula a partir de la pérdida `cur_loss`?
21. ¿Cómo se implementa el scheduler de tasa de aprendizaje (`StepLR`) y qué efecto tiene `gamma=0.9` cada 10000 pasos?
22. Al graficar las listas `Train_losses` y `Val_losses`, ¿qué información visual obtenemos y cómo la interpretarías para decidir si el modelo está sobreajustando?
23. Compara brevemente la generación de texto con el modelo GPT2 preentrenado (último bloque) frente al modelo `CustomGPTModel`: ¿en qué difieren los enfoques de tokenización y generación?
24. Identifica al menos dos puntos en los que podrías modularizar o refactorizar este código para mejorar su mantenibilidad.

In [ ]:
# Tus respuestas